In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import distance
from scipy.sparse.csgraph import minimum_spanning_tree
import math


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from modules.dp_func import *



In [4]:
# Set the mean and covariance matrix for each Gaussian distribution


mean1 = [0, 0, 0 , 0 , 0, 0, 0 , 0]
# covariance1 = np.ones((3,3))
covariance1 = np.identity(8)
# covariance1 = [[1, 0, 0],  [0, 1, 0], [0, 0, 0]

mean2 = [2.56, 0, 0 , 0, 0, 0,0, 0 ]
# mean2 = [math.sqrt(1/2), math.sqrt(1/2), 0]

# covariance2 = np.ones((3,3))
covariance2= np.identity(8)

#choose the number of samples per group. 
samples = 200

n0 = samples
n1 = samples

# p, q = .5, .5

In [5]:

bounds_l = []
bounds_u = []

MC_num = 500

for i in range(MC_num):
    # Generate random samples from the multivariate normal distributions
    data1 = np.random.multivariate_normal(mean1, covariance1, n0)
    data2 = np.random.multivariate_normal(mean2, covariance2, n1)

    
    FR = get_FR(data1, data2) #get the Frieman Rafsky statistic. 

    Dp = 1 - FR * (n0 + n1)/ (2 * n0 * n1)
    
    Up = Dp # we have choosen p =q
    if Up <= 0:
        print(i, FR, Up)
        analyze(data1, data2)
        
        
    lower, upper = calc_bounds(Up)
    
#     print("1, ", 1/2 - 1/2 * math.sqrt(Dp)  , FR/ (n0+ n1))
#     print("2, ", lower, upper)
    
    bounds_l.append(lower)
    bounds_u.append(upper)
    

print("The Bounds")

lower = np.average(bounds_l)
upper = np.average(bounds_u)
print(lower)
print(upper)

The Bounds
0.0990919418310319
0.17808500000000002


In [6]:
print(min(bounds_l))
print(max(bounds_l))


print(min(bounds_u))
print(max(bounds_u))

0.06411010564593267
0.15539878119774447
0.12
0.2625


In [7]:
ub = stats.describe(bounds_u)

DescribeResult(nobs=500, minmax=(0.12, 0.2625), mean=0.17808500000000002, variance=0.0005829361472945892, skewness=0.3676583925132904, kurtosis=0.04376503047446034)

In [11]:
dx = 0.001
x = np.arange(-5, 5 + dx, dx)

f0 = np.exp(-0.5 * (x - 0)**2) / np.sqrt(2 * np.pi)
f1 = np.exp(-0.5 * (x - 2.56)**2) / np.sqrt(2 * np.pi)

fmin = np.minimum(f0, f1)

BER = 0.5 * np.sum(fmin * dx)

print("BER 2d:", BER)

BER 2d: 0.10027244449669406


In [12]:
BER_lst = np.ones(MC_num) * BER


mse_l = ((BER_lst - bounds_l)**2).mean(axis=0)

mse_u =  ((BER_lst - bounds_u)**2).mean(axis=0)

print(mse_l, mse_u)

0.0002316224817309382 0.006636564068955067
